<a href="https://colab.research.google.com/github/QiSiyu/Delta-Modelling-ANN/blob/main/Train_DSM2_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

## Mount Google Drive

In [3]:
from google.colab import drive
import pandas as pd
import os
import glob
import numpy as np
import tensorflow as tf
import time
import sys
import functools
from pathlib import Path

google_drive_dir = 'DeltaModelling'

drive.mount('/content/drive',force_remount=True)
sys.path.append(os.path.join('/content/drive','My Drive',google_drive_dir))

data_resolution = '1D' # '1D' : daily; '15min' : 15-minute
key_stations = ['RSAN018', 'RSAC092', 'CHSWP003', 'CHDMC006', 'SLMZU025', 'ROLD059', 'CHVT000']


# daily data: <= 7306 samples, with 1282 input variables & 25 stations
# 15-min data: TODO

Mounted at /content/drive


## Define hyper-parameters

In [4]:
epochs = 1
batch_size = 100
initial_lr = 0.01
num_daily_values = 8
avg_window_size = 11
num_windows = 10
training_set_ratio = 0.8

window_size = num_windows * avg_window_size + num_daily_values

# True: convolutional layer is trainable; False: conv layer is fixed
train_conv = True


# create a folder to save trained models
model_folder = 'saved_models'

Path(os.path.join('/content/drive','My Drive',google_drive_dir,model_folder)).mkdir(parents=True, exist_ok=True)

# Load Data

In [5]:
data_files=sorted(tf.io.gfile.glob(os.path.join('/content/drive','My Drive',google_drive_dir,'data_%s_*.csv')%data_resolution))

fline=open(data_files[0]).readline().rstrip() # Reading column names 
variables=fline.split(',')

input_variables = [var for var in variables if 'input' in var]
output_variables = [var for var in variables if 'target' in var]
selected_input_variables = input_variables[0:]


DATASET_SIZE = -len(data_files) # number reserved for headers

for data_file in data_files:
    for row in open(data_file):
        DATASET_SIZE += 1
train_size = int(0.8 * DATASET_SIZE)
print('%d samples, each has %d input variable(s) & %d output station(s)' % (DATASET_SIZE, len(input_variables),len(output_variables)))


7306 samples, each has 1282 input variable(s) & 25 output station(s)


## Data preprocessing functions

In [6]:
def pack_features_vector(features,ec):
    """Pack the features into a single array."""
    features = tf.stack([tf.cast(x,tf.float32) for x in list(features.values())], axis=1)
    if ec.dtype==tf.float32:
        return features, ec
    else:
        return features, tf.strings.to_number(ec) if not tf.reduce_any(ec== b'') else tf.convert_to_tensor([float('NaN')],dtype=tf.float32)

def apply_window(dataset, window_size, batch_size,training_set_ratio=0.8):
    windowed_dataset = dataset.map(pack_features_vector)
    windowed_dataset = windowed_dataset.flat_map(lambda x, y: tf.data.Dataset.from_tensor_slices((x, y)))
    filter_nan_in_ec = lambda x, y: not tf.reduce_any(tf.math.logical_or(tf.math.is_nan(y),tf.math.less_equal(y,tf.constant([0],dtype=tf.float32))))
    windowed_dataset = windowed_dataset.filter(filter_nan_in_ec)

    num_samples = 0
    for _ in windowed_dataset:
        num_samples += 1
    train_size = int(training_set_ratio*num_samples)
    if num_samples - train_size < window_size:
        print('Available dataset size (%d training, %d test) smaller than window size (%d), will skip' % (train_size, num_samples - train_size,window_size))
        return None, None, 0, 0

    windowed_dataset = windowed_dataset.window(window_size, shift=1, drop_remainder=True)
    windowed_dataset = windowed_dataset.flat_map(lambda x, y: tf.data.Dataset.zip((x.batch(window_size), y)))
 
    windowed_trainset = windowed_dataset.take(train_size)
    windowed_testset = windowed_dataset.skip(train_size)

    return windowed_trainset.batch(batch_size).repeat(),\
        windowed_testset.batch(batch_size).repeat(),\
        train_size,\
        num_samples-train_size

In [7]:
def load_and_window_dataset(data_files, input_variables, output_variable, window_size,batch_size,training_set_ratio=0.8):
    assert len(output_variable)==1, 'This script is for single-station estimation!'

    # 1282 input variables, 25 salinity (output) stations
    ec_csv_ds = tf.data.experimental.make_csv_dataset(
        data_files,
        batch_size=1,
        na_value='-2.000', # missing entries in output
        select_columns=input_variables+output_variable,
        label_name=output_variable[0],# work on first output station
        num_epochs=1,
        ignore_errors=False,
        shuffle=False)

    windowed_trainset,windowed_testset,train_size,test_size = apply_window(ec_csv_ds, window_size, batch_size,training_set_ratio)

    #     for feature, label in windowed_testset.take(1):
    #         print('%s windowed feature shape: ' % (output_variable[0]), feature.shape, 'Label shape: ', label.shape)

    return windowed_trainset, windowed_testset, train_size,test_size


# Build model

In [8]:
# function that generates manual weights for first conv layer
def conv_filter_generator(num_daily_values=7,avg_window_size = 11, num_windows=10):
    w = np.zeros((1,num_daily_values+num_windows*avg_window_size,num_daily_values+num_windows))
    for ii in range(num_daily_values):
        w[0,num_daily_values+num_windows*avg_window_size-ii-1,num_daily_values-ii-1] = 1
    for ii in range(num_windows):
        w[0,((num_windows-ii-1)*avg_window_size):((num_windows-ii)*avg_window_size),num_daily_values+ii] = 1/avg_window_size
    return w

# weights to initialize the first conv layer
conv_filter_init = tf.keras.initializers.Constant(conv_filter_generator(num_daily_values=num_daily_values,
                                                                avg_window_size=avg_window_size,
                                                                num_windows=num_windows))


In [9]:
def build_ann(input_shape,num_daily_values, num_windows, conv_filter_init=None,train_conv=True,print_details=False):
    conv_layer = tf.keras.layers.Conv1D(num_daily_values+num_windows, 1, activation=None,
                              kernel_initializer=conv_filter_init,
                              kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0, l2=0))
    conv_layer.trainable=train_conv
    
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Permute((2,1))(inputs)
    x = tf.keras.layers.Masking(mask_value=-1)(x)
    x = conv_layer(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(10, activation="elu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(10, activation="elu")(x)
    # x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(1)(x)
    outputs = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="ann_model")
    if print_details:
        model.summary()
    return model

## Define Callback functions


### Learning rate scheduler

In [10]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 40%, 60%, 80%, 90% of total epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = initial_lr
    if epoch > 0.9*epochs:
        lr *= 0.5e-3
    elif epoch > 0.8*epochs:
        lr *= 1e-3
    elif epoch > 0.6*epochs:
        lr *= 1e-2
    elif epoch > 0.4*epochs:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
  
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

### Learning rate reducer

In [11]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1),
                                                 cooldown=0,
                                                 patience=5,
                                                 min_lr=initial_lr/100)

### Custom Loss Function

In [12]:
def mse_loss_masked(y_true, y_pred):
    squared_diff = tf.reduce_sum(tf.math.squared_difference(y_pred,y_true))
    return squared_diff/(tf.reduce_sum(tf.cast(y_true>0, tf.float32))+0.01)

# Compile and train model

In [13]:
losses = {}
val_losses = {}
ann_number = 0
for ii in range(len(output_variables)):
    if output_variables[ii].replace('target','') not in key_stations:
        pass
    ann_number += 1
    print('Training ANN #%d for Station %s' % (ann_number, output_variables[ii].replace('target','')))
    
    path_checkpoint = os.path.join('/content/drive','My Drive',google_drive_dir,model_folder,"%s_checkpoint.h5" % (output_variables[ii].replace('target','') + ('_train_conv' if train_conv else '_freeze_conv')))
    # es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)
    
    modelckpt_callback = tf.keras.callbacks.ModelCheckpoint(
        monitor="val_loss",
        filepath=path_checkpoint,
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    # prepare dataset
    selected_output_variable = output_variables[ii] # one output station at a time
    windowed_trainset, windowed_testset, train_size, test_size = load_and_window_dataset(data_files, 
                                                                                         selected_input_variables,
                                                                                         [selected_output_variable],
                                                                                         window_size,
                                                                                         batch_size,
                                                                                         training_set_ratio)
    if train_size == 0 or test_size == 0:
        continue

    # build new model
    model = build_ann(input_shape=(window_size, len(selected_input_variables)),
                      num_daily_values=num_daily_values,
                      num_windows=num_windows,
                      conv_filter_init=conv_filter_init,
                      train_conv=train_conv)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    
    model.compile(
        loss=mse_loss_masked,
        optimizer=optimizer,
        metrics=[tf.keras.metrics.MeanSquaredError()])

    history = model.fit(
      windowed_trainset,
      epochs=epochs,
      steps_per_epoch=train_size//batch_size,
      validation_data=windowed_testset,
      validation_freq=1,
      validation_steps=test_size//batch_size,
      callbacks=[lr_scheduler,modelckpt_callback],
      verbose=1
    )
    losses[selected_output_variable] = history.history['loss'][-1]
    val_losses[selected_output_variable] = history.history['val_loss'][-1]


Training ANN #1 for Station CHDMC006
Learning rate:  0.01
57/57 [==============================] - ETA: 0s - loss: 0.1124 - mean_squared_error: 0.1124
Epoch 00001: val_loss improved from inf to 0.14037, saving model to /content/drive/My Drive/DeltaModelling/saved_models/CHDMC006_train_conv_checkpoint.h5
57/57 [==============================] - 40s 669ms/step - loss: 0.1124 - mean_squared_error: 0.1124 - val_loss: 0.1404 - val_mean_squared_error: 0.1404 - lr: 0.0100
Training ANN #2 for Station CHSWP003
Learning rate:  0.01
54/54 [==============================] - ETA: 0s - loss: 0.0682 - mean_squared_error: 0.0682
Epoch 00001: val_loss improved from inf to 0.05419, saving model to /content/drive/My Drive/DeltaModelling/saved_models/CHSWP003_train_conv_checkpoint.h5
54/54 [==============================] - 41s 722ms/step - loss: 0.0682 - mean_squared_error: 0.0682 - val_loss: 0.0542 - val_mean_squared_error: 0.0542 - lr: 0.0100
Training ANN #3 for Station CHVCT000
Learning rate:  0.01
36

In [14]:
print (("{:<15}" * (3)).format('', 'Loss', 'Val Loss'))

for station, loss in losses.items():
    print(("{:<15}" * (3)).format(station.replace('target',''), np.round(loss,8), np.round(val_losses[station],8)))

               Loss           Val Loss       
CHDMC006       0.11239391     0.14037049     
CHSWP003       0.06823615     0.05418694     
CHVCT000       0.27601081     2.08799958     
OLD_MID        0.15451789     0.06863814     
ROLD024        0.1176272      0.18330817     
ROLD059        0.24607235     0.21902433     
RSAC064        0.10359459     0.95300621     
RSAC075        0.14228572     0.13725588     
RSAC081        0.1986206      0.05799089     
RSAC092        0.03310212     0.05390758     
RSAC101        0.12860398     0.03379159     
RSAN007        0.3759791      9.63587093     
RSAN018        0.11260133     0.10739285     
RSAN032        0.07800633     0.81832427     
RSAN037        0.10235355     0.32061225     
RSAN058        0.23551987     1.45540702     
RSAN072        0.16382222     0.155222       
RSMKL008       0.16483136     0.32964095     
SLCBN002       0.0850665      0.96922451     
SLDUT007       0.23848146     0.20616595     
SLMZU011       0.22181927     0.04